In [1]:
!python -m venv colab_website_venv
!source colab_website_venv/bin/activate


In [2]:
# 安装依赖
#%pip -q install fastapi uvicorn sqlalchemy "passlib[bcrypt]" cryptography pillow zipstream-new pyngrok nest_asyncio --upgrade



In [ ]:
# 配置云雾API密钥（必填，否则视频生成会失败）
import os
os.environ["YUNWU_API_KEY"] = "sk-LRZGHDhEJzL2DKPJ4DHtz3tfCfU8VQzymAtKvuI9WyspsVFh"
print("✅ YUNWU_API_KEY 已设置")


✅ YUNWU_API_KEY 已设置


In [ ]:
# 启动FastAPI服务并通过ngrok暴露公网地址
import os
import nest_asyncio
import uvicorn
from pyngrok import ngrok

# 确保不禁用后台任务队列（生产环境必须启用）
if "DISABLE_BACKGROUND" in os.environ:
    del os.environ["DISABLE_BACKGROUND"]

# Colab端口固定为 8888
port = int(os.environ.get("PORT", 8888))
# 打开一个 HTTP 隧道
public_tunnel = ngrok.connect(port, proto="http")
public_url = getattr(public_tunnel, "public_url", str(public_tunnel))
# 添加参数以跳过 ngrok 的浏览器警告页面
public_url_no_warn = f"{public_url}?ngrok-skip-browser-warning=true"
print("✅ Public URL:", public_url_no_warn)
os.environ["PUBLIC_BASE_URL"] = public_url
print("✅ PUBLIC_BASE_URL 已设置（带图生成可用）")

# 打印关键配置状态
if os.environ.get("YUNWU_API_KEY"):
    print("✅ YUNWU_API_KEY 已配置")
else:
    print("❌ 警告：YUNWU_API_KEY 未设置，视频生成将失败！")

# 解决在Jupyter事件循环中的运行问题
nest_asyncio.apply()

# 设置工作目录到项目根以便导入
import sys
sys.path.append(os.getcwd())

# 🔄 强制重新加载 settings 模块（解决 Colab 中模块缓存问题）
import importlib
if 'server.settings' in sys.modules:
    importlib.reload(sys.modules['server.settings'])
    print("✅ 已重新加载 settings 模块")

# 启动应用，需要在后台线程运行
config = uvicorn.Config("server.app:app", host="0.0.0.0", port=port, log_level="info")
server = uvicorn.Server(config)
await server.serve()



✅ Public URL: https://0df52634e912.ngrok-free.app?ngrok-skip-browser-warning=true
✅ PUBLIC_BASE_URL 已设置（带图生成可用）
✅ YUNWU_API_KEY 已配置


/opt/anaconda3/lib/python3.9/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.generics:GenericModel` has been moved to `pydantic.BaseModel`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
INFO:     Started server process [99719]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8888 (Press CTRL+C to quit)


[startup] Background executor starting...
[startup] Cleanup loop started. Public dir: /Users/zyma/Projects/ecommerce_video_generation_ai/ai_batch_generation_website/public
INFO:     74.211.102.81:0 - "GET /?ngrok-skip-browser-warning=true HTTP/1.1" 200 OK
INFO:     74.211.102.81:0 - "GET /api/me HTTP/1.1" 401 Unauthorized
INFO:     74.211.102.81:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     74.211.102.81:0 - "POST /api/login HTTP/1.1" 200 OK
INFO:     74.211.102.81:0 - "GET /api/me HTTP/1.1" 200 OK
INFO:     74.211.102.81:0 - "GET /api/batches?page=1&page_size=5 HTTP/1.1" 200 OK
INFO:     74.211.102.81:0 - "GET /api/batches?page=1&page_size=5 HTTP/1.1" 200 OK
INFO:     74.211.102.81:0 - "GET /api/batches/fcca2143-a433-408d-aa44-0769a287b514/tasks HTTP/1.1" 200 OK
[download_zip] 下载远程视频: https://filesystem.site/cdn/20251021/cbc2360b97b53a83cd5c454bb52759.mp4
INFO:     74.211.102.81:0 - "GET /api/batches?page=1&page_size=5 HTTP/1.1" 200 OK
INFO:     74.211.102.81:0 - "GET /api/b

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [99719]


In [ ]:
# ==========================================
# 💎 积分管理工具
# ==========================================
# 使用此 Cell 可以查看和调整用户积分

from server.db import SessionLocal
from server.models import User, CreditTransaction

def get_user_credits(user_id):
    """计算用户当前积分"""
    db = SessionLocal()
    try:
        total = db.query(CreditTransaction).filter(CreditTransaction.user_id == user_id).all()
        return sum(t.delta for t in total)
    finally:
        db.close()

def list_all_users():
    """列出所有用户及其积分"""
    db = SessionLocal()
    try:
        users = db.query(User).order_by(User.username).all()
        print("\n" + "="*60)
        print("当前所有用户的积分情况:")
        print("="*60)
        print(f"{'用户名':<15} {'权限':<10} {'实际积分':<10} {'User ID':<10}")
        print("-"*60)
        for user in users:
            role = "管理员" if user.is_admin else "普通用户"
            credits = get_user_credits(user.id)
            print(f"{user.username:<15} {role:<10} {credits:<10} {user.id:<10}")
        print("="*60 + "\n")
    finally:
        db.close()

def adjust_credits(username, delta, reason="管理员调整"):
    """调整用户积分"""
    db = SessionLocal()
    try:
        user = db.query(User).filter(User.username == username).first()
        if not user:
            print(f"❌ 用户 '{username}' 不存在!")
            return
        
        old_credits = get_user_credits(user.id)
        
        tx = CreditTransaction(
            user_id=user.id,
            delta=delta,
            reason=reason
        )
        db.add(tx)
        db.commit()
        
        new_credits = get_user_credits(user.id)
        
        action = "增加" if delta > 0 else "减少"
        abs_delta = abs(delta)
        print(f"\n✅ 积分调整成功:")
        print(f"   用户名: {username}")
        print(f"   操作: {action} {abs_delta} 积分")
        print(f"   原因: {reason}")
        print(f"   调整前: {old_credits} 积分")
        print(f"   调整后: {new_credits} 积分\n")
    except Exception as e:
        print(f"❌ 调整积分失败: {e}")
        db.rollback()
    finally:
        db.close()

# ==========================================
# 使用示例（取消注释来运行）:
# ==========================================

# 1. 查看所有用户及其积分
list_all_users()

# 2. 增加积分
# adjust_credits("doll", 100, "充值")

# 3. 减少积分
# adjust_credits("doll", -50, "扣款")

# 4. 批量操作示例
# adjust_credits("yimanxing", 200, "活动赠送")
# adjust_credits("test0", 50, "新手礼包")
# list_all_users()  # 再次查看确认
